In [1]:
import scipy.io as sp
import pandas as pd
import numpy as np

In [2]:
classe1 = sp.loadmat ('Classe1.mat')  #carrega a base de dados Classe1
classe2 = sp.loadmat ('Classe2.mat')  #carrega a base de dados Classe2
classe1 = pd.DataFrame.from_dict(classe1['Classe1']) #as operações acima carregam as bases em formas de array, e essas duas linhas convertem para tabelas dataframe
classe2 = pd.DataFrame.from_dict(classe2['Classe2'])

In [3]:
classe1 = classe1.T #pega a transposta
classe2 = classe2.T 
c1 = [1]*50         #cria um vetor de 50 posições preenchidas com o valor 1
c2 = [2]*50         #cria um vetor de 50 posições preenchidas com o valor 2
frames = [classe1, classe2]
baseA = pd.concat(frames) #concatena as duas bases em apenas um dataframe
base = pd.DataFrame()     #cria um dataframe vazio

In [4]:
base["mean"] = baseA.mean(axis=1)   #cria uma coluna "mean" e preenche com as medias de cada linha
base["median"] = baseA.median(axis=1)  #cria uma coluna "median" e preenche com as medianas de cada linha
base["std"] = baseA.std(axis=1)        #cria uma coluna "std" e preenche com os desvio padrão de cada linha
base["max"] = baseA.max(axis=1)        #cria uma coluna "max" e preenche com os valores maximos de cada linha
base["skewness"] = baseA.skew(axis=1)  #cria uma coluna "skewness" e preenche com a assimetria de cada linha
base["labels"] = c1+c2                 #atribui os rotulos a cada linha
base = base.reset_index(drop=True)     #reseta os indices do dataframe para ir de 0 a 99

In [6]:
acuracia = 0                                                               #variável para acumular os acertos
for K in range(10):                                                        #for para variar o K-fold
    if (K == 0):
        test = base.loc[0:9]
        train = base.loc[10:99]
    elif (K == 9):
        test = base.loc[K*10:(K*10)+9]
        train = base.loc[0:89]
    else:
        test = base.loc[K*10:(K*10)+9]
        train = pd.concat([base.loc[0:(K*10)-1], base.loc[(K*10)+10:99]])
    train = train.reset_index(drop=True)                                  #reseta os indices do dataframe para ir de 0 a 89
    test = test.reset_index(drop=True)                                    #reseta os indices do dataframe para ir de 0 a 9
    centros = train.groupby(["labels"]).mean()                            #calcula a media de cada atributo de cada classe
    centros = centros.reset_index(drop=True) 
    for d in range(10):                                                   #percorre as 10 amostras
        amostra = test.iloc[d,:-1]                                        #seleciona a amostra de test da posição 'd'
        distancia = pd.DataFrame({"distancia":[],"labels":[]})            #cria um dataframe vazio
        for a in range(2):                                               #percorre as 90 linhas do grupo train
            b = [np.sqrt(sum(pow(centros.iloc[a,:].T - amostra.T,2))), a+1 ]  #calcula a distancia euclidiana da amosta[d] com o train[a]
            distancia = distancia.append({'distancia':b[0],'labels':b[1]},ignore_index=True) #adiciona uma linha com a distancia e com o rotulo de train[a]
        distancia = distancia.sort_values('distancia')[0:1]                                  #ordena as distancias em ordem crescente e seleciona os k primeiros
        if(test['labels'][d] == distancia["labels"].mode().values[0]):                              #testa se o rotulo é o mesmo da moda das k primeiras distancias
            acuracia += 1
print("A acurácia para é ", acuracia/100)

A acurácia para é  0.79
